<a href="https://colab.research.google.com/github/EricSiq/India_Missing_Persons_Analysis_2017-2022/blob/main/ClusteringUMLProject2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Kaggle Dataset: 5 Years Districtwise India Missing Person's Dataset](https://www.kaggle.com/datasets/ericsiq/india-5-years-districtwise-missing-persons-dataset)


[GitHub Repo](https://github.com/EricSiq/India_Missing_Persons_Analysis_2017-2022)

In [201]:
!pip install tabulate



# Overview:
# This notebook demonstrates how to load multiple CSV datasets spanning the years, 2017 to 2022, merge them into a single dataframe, and perform several data processing and exploratory data analysis (EDA) steps. The sections below cover:







In [202]:
# Importing libraries
import pandas as pd         # For data manipulation and analysis
import numpy as np          # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs
import seaborn as sns       # For enhanced visualization
from tabulate import tabulate #For tabular outputs



# Data Loading



> Upon loading of the datasets, it is noticed there is a serious disrepancy between column values for age groups.



The age group classifications differ notably between the 2018–2020 and 2021–2022 datasets:


2018–2020: Age brackets are more granular and traditional:

Below 5 years

5–14 years

14–18 years

18–30 years

30–45 years

45–60 years

60 years & above



In 2021–2022: The classification structure has changed:

Below 12 years

12–16 years

16–18 years

18 years & above

In [203]:
%matplotlib inline

# Setting a style for seaborn plots
sns.set(style="whitegrid")

In [204]:

# Section 1: Define file paths for each year's data.
file_paths = {
    2018: "/content/DistrictwiseMissingPersons2018.csv",
    2019: "/content/DistrictwiseMissingPersons2019.csv",
    2020: "/content/DistrictwiseMissingPersons2020.csv",
    2021: "/content/DistrictwiseMissingPersons2021.csv",
    2022: "/content/DistrictwiseMissingPersons2022.csv"
}




> Due to disrepancies between columns, we need to remove unnessessary columns and make them uniform across 2018-2022 years.

> We have to group all age groups into either Children or 18+ age groups to simplify the age groups.



#   a) Data Loading:
1.     - Reading the CSV files into pandas dataframes.
2.     - Removing Unnessessary Column values
3.     - Examining initial structure & description of the data.

In [205]:

# A list to hold all processed DataFrames.
dfs = []

# Section 2: Process each dataset according to its year.
for year, path in file_paths.items():
    # Load file with fallback encoding if necessary
    try:
        df = pd.read_csv(path)
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(path, encoding='ISO-8859-1')
            print(f"Used fallback encoding for {year}")
        except Exception as e:
            print(f"Failed to load {year}: {e}")
            continue

    # Add the year column if not already present.
    df['Year'] = year

    # Remove any leading/trailing whitespace from column headers.
    df.columns = df.columns.str.strip()

    if year <= 2020:
        # For datasets 2018-2020, we have the detailed age-group columns.
        # Male columns
        male_below_18 = [
            'Male_Below_5_years',
            'Male_5_years_&_Above_Below_14_years',
            'Male_14_years_&_Above_Below_18_years'
        ]
        male_above_18 = [
            'Male_18_years_&_Above_Below_30_years',
            'Male_30_years_&_Above_Below_45_years',
            'Male_45_years_&_Above_Below_60_years',
            'Male_60_years_&_Above'
        ]

        # Female columns
        female_below_18 = [
            'Female_Below_5_years',
            'Female_5_years_&_Above_Below_14_years',
            'Female_14_years_&_Above_Below_18_years'
        ]
        female_above_18 = [
            'Female_18_years_&_Above_Below_30_years',
            'Female_30_years_&_Above_Below_45_years',
            'Female_45_years_&_Above_Below_60_years',
            'Female_60_years_&_Above'
        ]

        # Transgender columns
        trans_below_18 = [
            'Transgender_Below_5_years',
            'Transgender_5_years_&_Above_Below_14_years',
            'Transgender_14_years_&_Above_Below_18_years'
        ]
        trans_above_18 = [
            'Transgender_18_years_&_Above_Below_30_years',
            'Transgender_30_years_&_Above_Below_45_years',
            'Transgender_45_years_&_Above_Below_60_years',
            'Transgender_60_years_&_Above'
        ]

        # Total columns
        total_below_18 = [
            'Total_Below_5_years',
            'Total_5_years_&_Above_Below_14_years',
            'Total_14_years_&_Above_Below_18_years'
        ]
        total_above_18 = [
            'Total_18_years_&_Above_Below_30_years',
            'Total_30_years_&_Above_Below_45_years',
            'Total_45_years_&_Above_Below_60_years',
            'Total_60_years_&_Above'
        ]

        # Sum up the relevant columns for each group.
        df['Male_Below_18'] = df[male_below_18].sum(axis=1)
        df['Male_18_and_above'] = df[male_above_18].sum(axis=1)

        df['Female_Below_18'] = df[female_below_18].sum(axis=1)
        df['Female_18_and_above'] = df[female_above_18].sum(axis=1)

        df['Transgender_Below_18'] = df[trans_below_18].sum(axis=1)
        df['Transgender_18_and_above'] = df[trans_above_18].sum(axis=1)

        df['Total_Below_18'] = df[total_below_18].sum(axis=1)
        df['Total_18_and_above'] = df[total_above_18].sum(axis=1)

        # Drop the original detailed columns.
        drop_cols = (male_below_18 + male_above_18 +
                     female_below_18 + female_above_18 +
                     trans_below_18 + trans_above_18 +
                     total_below_18 + total_above_18)
        df.drop(columns=drop_cols, inplace=True, errors='ignore')

    else:
        # For 2021-2022, the files already include aggregated age-group columns.
        # Rename them to standardized names.
        rename_map = {
            'Male_Children': 'Male_Below_18',
            'Male_18_years_&_Above': 'Male_18_and_above',
            'Female_Children': 'Female_Below_18',
            'Female_18_years_&_Above': 'Female_18_and_above',
            'Transgender_Children': 'Transgender_Below_18',
            'Transgender_18_years_&_Above': 'Transgender_18_and_above',
            'Total_Children': 'Total_Below_18',
            'Total_18_years_&_Above': 'Total_18_and_above'
        }
        df.rename(columns=rename_map, inplace=True)

        # Drop any extra detailed age-group columns that are not needed.
        drop_cols = [
            'Male_Below_12_years', 'Male_12_years_&_Above_Below_16_years', 'Male_16_years_&_Above_Below_18_years',
            'Female_Below_12_years', 'Female_12_years_&_Above_Below_16_years', 'Female_16_years_&_Above_Below_18_years',
            'Transgender_Below_12_years', 'Transgender_12_years_&_Above_Below_16_years', 'Transgender_16_years_&_Above_Below_18_years',
            'Total_Below_12_years', 'Total_12_years_&_Above_Below_14_years', 'Total_14_years_&_Above_Below_18_years'
        ]
        df.drop(columns=drop_cols, inplace=True, errors='ignore')

    # Append the processed DataFrame to our list.
    dfs.append(df)
    print(f"Loaded and processed data for {year} with shape: {df.shape}")

# Optionally, display a preview of the first processed DataFrame.
print("\nPreview of the processed dataset for the first file:")
print(tabulate(dfs[0].head(10), headers='keys', tablefmt='pretty'))

Loaded and processed data for 2018 with shape: (892, 15)
Loaded and processed data for 2019 with shape: (912, 15)
Loaded and processed data for 2020 with shape: (932, 15)
Loaded and processed data for 2021 with shape: (941, 15)
Loaded and processed data for 2022 with shape: (969, 15)

Preview of the processed dataset for the first file:
+---+------+----------------+------------------+------------+--------------+-------------------+-------------+---------------+-------------------+-----------------+---------------------+----------------------+--------------------------+----------------+--------------------+
|   | Year |     State      |     District     | Total_Male | Total_Female | Total_Transgender | Grand_Total | Male_Below_18 | Male_18_and_above | Female_Below_18 | Female_18_and_above | Transgender_Below_18 | Transgender_18_and_above | Total_Below_18 | Total_18_and_above |
+---+------+----------------+------------------+------------+--------------+-------------------+-------------+-

In [206]:
print(tabulate(dfs[1].head(10), headers='keys', tablefmt='pretty'))

+---+------+----------------+------------------+------------+--------------+-------------------+-------------+---------------+-------------------+-----------------+---------------------+----------------------+--------------------------+----------------+--------------------+
|   | Year |     State      |     District     | Total_Male | Total_Female | Total_Transgender | Grand_Total | Male_Below_18 | Male_18_and_above | Female_Below_18 | Female_18_and_above | Transgender_Below_18 | Transgender_18_and_above | Total_Below_18 | Total_18_and_above |
+---+------+----------------+------------------+------------+--------------+-------------------+-------------+---------------+-------------------+-----------------+---------------------+----------------------+--------------------------+----------------+--------------------+
| 0 | 2019 | Andhra Pradesh |    Anantapur     |    257     |     766      |         0         |    1023     |      60       |        197        |       280       |        486

In [207]:
print(tabulate(dfs[2].head(10), headers='keys', tablefmt='pretty'))

+---+------+----------------+---------------+------------+--------------+-------------------+-------------+---------------+-------------------+-----------------+---------------------+----------------------+--------------------------+----------------+--------------------+
|   | Year |     State      |   District    | Total_Male | Total_Female | Total_Transgender | Grand_Total | Male_Below_18 | Male_18_and_above | Female_Below_18 | Female_18_and_above | Transgender_Below_18 | Transgender_18_and_above | Total_Below_18 | Total_18_and_above |
+---+------+----------------+---------------+------------+--------------+-------------------+-------------+---------------+-------------------+-----------------+---------------------+----------------------+--------------------------+----------------+--------------------+
| 0 | 2020 | Andhra Pradesh |   Anantapur   |    209     |     869      |         0         |    1078     |      32       |        177        |       284       |         585         | 

In [208]:
print(tabulate(dfs[3].head(10), headers='keys', tablefmt='pretty'))

+---+------+----------------+---------------+------------+---------------+-------------------+--------------+-----------------+---------------------+-------------------+----------------------+--------------------------+-------------+----------------+--------------------+
|   | Year |     State      |   District    | Total_Male | Male_Below_18 | Male_18_and_above | Total_Female | Female_Below_18 | Female_18_and_above | Total_Transgender | Transgender_Below_18 | Transgender_18_and_above | Grand_Total | Total_Below_18 | Total_18_and_above |
+---+------+----------------+---------------+------------+---------------+-------------------+--------------+-----------------+---------------------+-------------------+----------------------+--------------------------+-------------+----------------+--------------------+
| 0 | 2021 | Andhra Pradesh |   Anantapur   |    291     |      43       |        248        |     1224     |       446       |         778         |         0         |          0    

In [209]:
print(tabulate(dfs[4].head(10), headers='keys', tablefmt='pretty'))

+---+------+----------------+--------------------------+------------+---------------+-------------------+--------------+-----------------+---------------------+-------------------+----------------------+--------------------------+-------------+----------------+--------------------+
|   | Year |     State      |         District         | Total_Male | Male_Below_18 | Male_18_and_above | Total_Female | Female_Below_18 | Female_18_and_above | Total_Transgender | Transgender_Below_18 | Transgender_18_and_above | Grand_Total | Total_Below_18 | Total_18_and_above |
+---+------+----------------+--------------------------+------------+---------------+-------------------+--------------+-----------------+---------------------+-------------------+----------------------+--------------------------+-------------+----------------+--------------------+
| 0 | 2022 | Andhra Pradesh |  Alluri Sitharama Raju   |     36     |       8       |        28         |      80      |       54        |         26  

#   b) Pre-processing:
1.      - Analysing dataset values
2.      - Merging the datasets.
2.     - Handling missing values and data type conversions.

In [210]:
print("Columns in the DataFrame:")
print(df.columns.tolist())

Columns in the DataFrame:
['Year', 'State', 'District', 'Total_Male', 'Male_Below_18', 'Male_18_and_above', 'Total_Female', 'Female_Below_18', 'Female_18_and_above', 'Total_Transgender', 'Transgender_Below_18', 'Transgender_18_and_above', 'Grand_Total', 'Total_Below_18', 'Total_18_and_above']


In [211]:
# Concatenate all dataframes into a single dataframe
data = pd.concat(dfs, ignore_index=True)

# Display combined dataframe shape and basic info
print("Combined dataset shape:", data.shape)
print("\nDataset Info:")
data.info()

# Check for missing values in each column
missing_values = data.isna().sum()
print("\nMissing Values per column:\n", missing_values)

Combined dataset shape: (4646, 15)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646 entries, 0 to 4645
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year                      4646 non-null   int64  
 1   State                     4646 non-null   object 
 2   District                  4646 non-null   object 
 3   Total_Male                4646 non-null   int64  
 4   Total_Female              4646 non-null   int64  
 5   Total_Transgender         4646 non-null   int64  
 6   Grand_Total               4646 non-null   int64  
 7   Male_Below_18             4646 non-null   int64  
 8   Male_18_and_above         4646 non-null   float64
 9   Female_Below_18           4646 non-null   int64  
 10  Female_18_and_above       4646 non-null   float64
 11  Transgender_Below_18      4646 non-null   int64  
 12  Transgender_18_and_above  4646 non-null   int64  
 13  Total_Below_1


*Handling Missing Values:*

  - Remove rows with missing key values.

In [215]:
# Display all rows that have at least one missing value
rows_with_missing = data[data.isna().any(axis=1)]
print("\nRows with missing values:")
print(rows_with_missing)

# Create a new DataFrame by removing rows with missing values
data_clean = data.dropna()




Rows with missing values:
Empty DataFrame
Columns: [Year, State, District, Total_Male, Total_Female, Total_Transgender, Grand_Total, Male_Below_18, Male_18_and_above, Female_Below_18, Female_18_and_above, Transgender_Below_18, Transgender_18_and_above, Total_Below_18, Total_18_and_above]
Index: []


#   c) Exploratory Data Analysis (EDA):
1.      - Statistical summary.
2.      - Distribution of key variables.
3.     - Trends across years and per district.
4.     - Visualizations with appropriate parameters.


In [213]:

# In the EDA section, we will:
#   - Look at the summary statistics using descriptive statistics.
#   - Visualize distributions of the key numerical variables.
#   - Analyze trends over years and compare distributions across different districts.
#   - Use visualizations (e.g., histograms, boxplots, line charts, heatmaps) with robust parameters.

# Display descriptive statistics for numerical columns
print("Descriptive statistics for numerical columns:")
display(data[num_cols].describe())

# Visualizing distribution of a key numerical variable
# Here, we assume that there is a column like 'MissingPersonsCount' (adjust the column name as necessary)
# If the exact column name is different, replace 'MissingPersonsCount' with the appropriate one.

if "MissingPersonsCount" in data.columns:
    plt.figure(figsize=(10,6))
    sns.histplot(data["MissingPersonsCount"], kde=True, bins=30)
    plt.title("Distribution of Missing Persons Count")
    plt.xlabel("Missing Persons Count")
    plt.ylabel("Frequency")
    plt.show()
else:
    print("Column 'MissingPersonsCount' not found in the dataset.")

# Example: Box plot to check the distribution per year or per district.
# We assume there is a district column named 'District'. Adjust as needed.

if "District" in data.columns and "MissingPersonsCount" in data.columns:
    plt.figure(figsize=(14,8))
    sns.boxplot(x="Year", y="MissingPersonsCount", data=data)
    plt.title("Boxplot of Missing Persons Count per Year")
    plt.xlabel("Year")
    plt.ylabel("Missing Persons Count")
    plt.show()
else:
    print("Required columns for boxplot ('District' or 'MissingPersonsCount') not found.")

# Analyzing trends per district over the years.
# Let's create a pivot table of average missing persons count by district and year.
if "District" in data.columns and "MissingPersonsCount" in data.columns:
    pivot_table = data.pivot_table(values="MissingPersonsCount",
                                   index="District",
                                   columns="Year",
                                   aggfunc="mean")
    print("Pivot Table of Average Missing Persons Count by District and Year:")
    display(pivot_table)

    # Plot a heatmap for visual representation
    plt.figure(figsize=(12,10))
    sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="YlGnBu")
    plt.title("Heatmap of Average Missing Persons Count by District and Year")
    plt.xlabel("Year")
    plt.ylabel("District")
    plt.show()
else:
    print("Required columns for pivot table ('District' or 'MissingPersonsCount') not found.")

# Correlation analysis: Visualize the correlation between numerical features.
plt.figure(figsize=(10,8))
corr_matrix = data[num_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap among Numerical Features")
plt.show()


Descriptive statistics for numerical columns:


NameError: name 'num_cols' is not defined

#  Additional EDA Parameters and Considerations:





# - **Parameter Choice in Visualizations:**  
#   We have chosen appropriate figure sizes, bin counts, and color maps to maximize the clarity of the plots.


# - **Handling Outliers:**  
#   The box plots can help identify potential outliers which might need further investigation.


# - **Time Trends:**  
#   Using pivot tables and heatmaps to study trends across different years and districts enables a comprehensive overview.


# - **Correlation Analysis:**  
#   Examining the correlation matrix helps us understand the relationships between various numerical variables.


# - **Adjustments:**  
#   Adjust column names or plot parameters as necessary once you confirm the actual structure of the dataset.
# This notebook template offers a comprehensive view of loading, pre-processing, and analyzing the data.
# Depending on the specifics of the dataset (column names, data quality, etc.), additional steps (e.g.,
# more advanced imputation techniques or outlier detection) may be required.